# Installation

In [1]:
! pip install "openai>=1.0.0,<2.0.0"
! pip install "azure-identity>=1.15.0"
! pip install pandas python-docx
! pip install scikit-learn==1.5.1
! pip install skops
import os
from openai import AzureOpenAI

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#Embeding

In [2]:
from google.colab import userdata
# Gateway setting to OpenAI API
client = AzureOpenAI(
    # This is the default and can be omitted
    #azure_endpoint = os.environ.get('ENDPOINT'),
    #api_key = os.environ.get('OPENAI_API_KEY'),
    azure_endpoint = userdata.get('ENDPOINT'),
    api_key = userdata.get('OPENAI_API_KEY'),
    api_version="2024-05-01-preview",
)

def generate_embeddings(text):
  embedResult = client.embeddings.create(input = text, model="text-embedding-ada-002")
  #print(embedResult)
  return embedResult.data[0].embedding



# Import Data Source

In [3]:
import pandas as pd
import json
dataDict = pd.DataFrame(columns=['dataSource', 'dataString'])
dataDict

path = '/content/drive/MyDrive/Colab Notebooks/chatbot/'
querySimilarity = pd.read_json(path+"querySimilarity.json")


path = '/content/drive/MyDrive/Colab Notebooks/chatbot/'
dataDict = pd.read_json(path+"dataDict.json")

# Initiate Data Source

In [ ]:
ClassSchedule_path = "https://docs.google.com/spreadsheets/d/1R3ptCqJdlHhW4ZP4KElZ1zdDmhRKs09s/export"
df = pd.read_excel(ClassSchedule_path)
for i in df:
  df["combined"] = "Week: " + df['Week'] + "; Section: " + df['Section'] + "; Date: " + df.Date.astype(str)+ "; Time: " + df.Time.astype(str) + "; Class: " + df.Class+';\n'
combineText = ' '.join(df["combined"])
if 'Class Schedule' in dataDict['dataSource'].values :
  dataDict.loc[dataDict['dataSource'] == "Class Schedule", 'dataString'] = combineText
else:
  temp = {'dataSource':'Class Schedule',
            'dataString':combineText}
  dataDict=dataDict._append(temp, ignore_index = True)
dataDict

instructorList_path = "https://docs.google.com/spreadsheets/d/1WQoDZ6S_xfbLra-qfwGswiLZa5bxqR-1/export"
df = pd.read_excel(instructorList_path)
for i in df:
  df["combined"] = "Section: " + df['Section'] + "; Instructor: " + df['Instructor'] + "; Email: " + df['email']+';\n'
combineText = ' '.join(df["combined"])
if 'Instructor Contacts' in dataDict['dataSource'].values :
  dataDict.loc[dataDict['dataSource'] == "Instructor Contacts", 'dataString'] = combineText
else:
  temp = {'dataSource':'Instructor Contacts',
            'dataString':combineText}
  dataDict=dataDict._append(temp, ignore_index = True)
dataDict


,dataSource,dataString,embeding,similarities
0,Class Schedule,Week: Week 1; Section: all sections; Date: 202...,"[-0.015398529400000001, 0.0026727747, 0.003457...",0.773265
1,Instructor Contacts,Section: T01 ; Instructor: Miss Karina Yuen; E...,"[-0.0177541431, -0.0163448658, -0.0047735851, ...",0.762478
2,Assessments and Grading,"{\n ""document_type"": ""Assessment Guide"",\n ""...","[-0.011931576800000001, 0.0221283957, -0.00337...",0.843341
3,Administrative Issues and Policies,"{\n ""document_type"": ""Admin FAQ"",\n ""course_...","[0.0017108611, 0.0127634853, -0.0262246523, -0...",0.779640
4,Course Content and Structure,"{\n ""course_id"": ""PDEV6800Z"",\n ""course_name...","[0.0049033058000000004, 0.0052766027, -0.00880...",0.790362
5,Gamification,"{\n ""document_type"": ""Gamification Guide"",\n ...","[-0.0155429756, 0.007033336, -0.0141363088, -0...",0.783072


In [ ]:
#Import modules
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

#Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

#Get the Shareable link
#Ex link : https://drive.google.com/file/d/1c7Ffo1Go1dtUpKcSWxdbdVyW4dfhEoUp/view?usp=sharing
# Get the id from the link 1c7Ffo1Go1dtUpKcSWxdbdVyW4dfhEoUp
downloaded = drive.CreateFile({'id':"1Cinn1LBaU3g6e1uGXA6xHQW6gN5Sj6gT"})
downloaded.GetContentFile('Assessment.docx')

downloaded = drive.CreateFile({'id':"1om4jb47oviqNZJ-YapEBuAU97c51CK_1"})
downloaded.GetContentFile('Admin FAQ.docx')

downloaded = drive.CreateFile({'id':"1yKvTMWi29Z-Nat45LEEJ6aC2Y6heIP7U"})
downloaded.GetContentFile('Course Content.docx')

downloaded = drive.CreateFile({'id':"1Rgj9EwqGPSoGLjkM9edFI7ox-9zY6DXP"})
downloaded.GetContentFile('Gamification.docx')

In [ ]:
from docx import Document

doc = Document('Assessment.docx')
full_text = []
for para in doc.paragraphs:
    full_text.append(para.text)
#print('\n'.join(full_text))
if 'Assessments and Grading' in dataDict['dataSource'].values :
  dataDict.loc[dataDict['dataSource'] == "Assessments and Grading", 'dataString'] = '\n'.join(full_text)
else:
  temp = {'dataSource':'Assessments and Grading',
            'dataString':'\n'.join(full_text)}
  dataDict=dataDict._append(temp, ignore_index = True)

doc = Document('Admin FAQ.docx')
full_text = []
for para in doc.paragraphs:
    full_text.append(para.text)
if 'Administrative Issues and Policies' in dataDict['dataSource'].values :
  dataDict.loc[dataDict['dataSource'] == "Administrative Issues and Policies", 'dataString'] = '\n'.join(full_text)
else:
  temp = {'dataSource':'Administrative Issues and Policies',
            'dataString':'\n'.join(full_text)}
  dataDict=dataDict._append(temp, ignore_index = True)

doc = Document('Course Content.docx')
full_text = []
for para in doc.paragraphs:
    full_text.append(para.text)
if 'Course Content and Structure' in dataDict['dataSource'].values :
  dataDict.loc[dataDict['dataSource'] == "Course Content and Structure", 'dataString'] = '\n'.join(full_text)
else:
  temp = {'dataSource':'Course Content and Structure',
            'dataString':'\n'.join(full_text)}
  dataDict=dataDict._append(temp, ignore_index = True)

doc = Document('Gamification.docx')
full_text = []
for para in doc.paragraphs:
    full_text.append(para.text)
if 'Gamification' in dataDict['dataSource'].values :
  dataDict.loc[dataDict['dataSource'] == "Gamification", 'dataString'] = '\n'.join(full_text)
else:
  temp = {'dataSource':'Gamification',
            'dataString':'\n'.join(full_text)}
  dataDict=dataDict._append(temp, ignore_index = True)

dataDict

,dataSource,dataString,embeding,similarities
0,Class Schedule,Week: Week 1; Section: all sections; Date: 202...,"[-0.015398529400000001, 0.0026727747, 0.003457...",0.773265
1,Instructor Contacts,Section: T01 ; Instructor: Miss Karina Yuen; E...,"[-0.0177541431, -0.0163448658, -0.0047735851, ...",0.762478
2,Assessments and Grading,"{\n ""document_type"": ""Assessment Guide"",\n ""...","[-0.011931576800000001, 0.0221283957, -0.00337...",0.843341
3,Administrative Issues and Policies,"{\n ""document_type"": ""Admin FAQ"",\n ""course_...","[0.0017108611, 0.0127634853, -0.0262246523, -0...",0.779640
4,Course Content and Structure,"{\n ""course_id"": ""PDEV6800Z"",\n ""course_name...","[0.0049033058000000004, 0.0052766027, -0.00880...",0.790362
5,Gamification,"s{\n ""document_type"": ""GamificatioN"",\n ""cou...","[-0.0155429756, 0.007033336, -0.0141363088, -0...",0.783072


#Generate Embedding

In [19]:
querySimilarity

,Query,class,embeding,cosine_pred,jaccard_pred,manhattan_pred,euclidean_pred
0,What is the overall duration of the PDEV6800Z ...,Class Schedule,"[0.0132006425, -0.0141715817, 0.00358765500000...",Administrative Issues and Policies,Class Schedule,Administrative Issues and Policies,Administrative Issues and Policies
1,How many weeks does the course span?,Class Schedule,"[0.0074779699, -0.0019203861, 0.0194210075, -0...",Class Schedule,Instructor Contacts,Class Schedule,Class Schedule
2,What are the different sections available for ...,Class Schedule,"[-0.0027097552, -0.0034730197, 0.0012607195, -...",Class Schedule,Instructor Contacts,Class Schedule,Class Schedule
3,How many modules are there in the course?,Class Schedule,"[9.09918e-05, -0.0010510976, 0.0036951634, -0....",Class Schedule,Instructor Contacts,Class Schedule,Class Schedule
4,What is the general structure of the course sc...,Class Schedule,"[0.0001531849, -0.0038599621000000002, 0.00647...",Class Schedule,Instructor Contacts,Class Schedule,Class Schedule
...,...,...,...,...,...,...,...
1095,Can you teach me how to juggle water?,irrelevant,"[-0.007677948600000001, -0.0051693441, 0.00894...",Instructor Contacts,Instructor Contacts,Instructor Contacts,Instructor Contacts
1096,What's the best way to butter toast underwater?,irrelevant,"[-0.0058801901, -0.0015551196, 0.0335660838, 0...",Instructor Contacts,Instructor Contacts,Instructor Contacts,Instructor Contacts
1097,How do I grow a money tree?,irrelevant,"[0.004561421, -0.0358112417, -0.0048262775, -0...",Gamification,Instructor Contacts,Gamification,Gamification
1098,Why don't we call oranges 'yellows'?,irrelevant,"[0.0218553264, -0.0252066515, 0.0213598013, -0...",Instructor Contacts,Instructor Contacts,Instructor Contacts,Instructor Contacts


In [18]:

dataDict

,dataSource,dataString,embeding,similarities
0,Class Schedule,Week: Week 1; Section: all sections; Venue: na...,"[-0.0109663289, -0.0020042628, 0.0042889146, -...",0.807086
1,Instructor Contacts,Section: T01 ; Instructor: Miss Karina Yuen; E...,"[-0.0177541431, -0.0163448658, -0.0047735851, ...",0.772297
2,Assessments and Grading,"{\n ""document_type"": ""Assessment Guide"",\n ""...","[-0.013315695300000001, 0.0365025364, -0.00063...",0.711249
3,Administrative Issues and Policies,"{\n ""document_type"": ""Admin FAQ"",\n ""course_...","[0.0069416948, 0.0098400135, -0.0265979376, -0...",0.752453
4,Course Content and Structure,"{\n ""course_id"": ""PDEV6800Z"",\n ""course_name...","[0.0025590127, 0.0049421252, -0.0091129951, -0...",0.717063
5,Gamification,"{\n ""document_type"": ""Gamification Guide"",\n ...","[-0.0069681741, 0.011704028600000001, -0.01288...",0.715762


In [ ]:
dataDict['embeding'] = dataDict.dataString.apply(lambda x : generate_embeddings(x))
dataDict.to_json(path+"dataDict.json")
dataDict

,dataSource,dataString,embeding,similarities
0,Class Schedule,Week: Week 1; Section: all sections; Venue: na...,"[-0.010966328904032707, -0.0020042627584189177...",0.807086
1,Instructor Contacts,Section: T01 ; Instructor: Miss Karina Yuen; E...,"[-0.017754143103957176, -0.016344865784049034,...",0.772297
2,Assessments and Grading,"{\n ""document_type"": ""Assessment Guide"",\n ""...","[-0.013315695337951183, 0.03650253638625145, -...",0.711249
3,Administrative Issues and Policies,"{\n ""document_type"": ""Admin FAQ"",\n ""course_...","[0.006941694766283035, 0.009840013459324837, -...",0.752453
4,Course Content and Structure,"{\n ""course_id"": ""PDEV6800Z"",\n ""course_name...","[0.002559012733399868, 0.004942125175148249, -...",0.717063
5,Gamification,"{\n ""document_type"": ""Gamification Guide"",\n ...","[-0.0069681741297245026, 0.011704028584063053,...",0.715762


In [ ]:
querySimilarity["embeding"] = querySimilarity.Query.apply(lambda x : generate_embeddings(x))
querySimilarity.to_json(path+"querySimilarity.json")
querySimilarity.to_json(path+"trainingQuery.json")
querySimilarity

,Query,class,embeding
0,What is the overall duration of the PDEV6800Z ...,Class Schedule,"[0.013200642541050911, -0.014171581715345383, ..."
1,How many weeks does the course span?,Class Schedule,"[0.007477969862520695, -0.0019203861011192203,..."
2,What are the different sections available for ...,Class Schedule,"[-0.0027097552083432674, -0.003473019693046808..."
3,How many modules are there in the course?,Class Schedule,"[9.099176531890407e-05, -0.0010510976426303387..."
4,What is the general structure of the course sc...,Class Schedule,"[0.0001531848538434133, -0.0038599621038883924..."
...,...,...,...
1095,Can you teach me how to juggle water?,irrelevant,"[-0.007677948568016291, -0.005169344134628773,..."
1096,What's the best way to butter toast underwater?,irrelevant,"[-0.005880190059542656, -0.0015551196411252022..."
1097,How do I grow a money tree?,irrelevant,"[0.004561420995742083, -0.03581124171614647, -..."
1098,Why don't we call oranges 'yellows'?,irrelevant,"[0.021855326369404793, -0.02520665153861046, 0..."


# Calculate Similarity

In [ ]:
#find the most relevant documents
import numpy as np
from math import sqrt
#cosine_similarity
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

#jaccard_similarity
def jaccard_similarity(a, b):
    set1=set(a)
    set2=set(b)
    # intersection of two sets
    intersection = len(set1.intersection(set2))
    # Unions of two sets
    union = len(set1.union(set2))
    return intersection / union

def manhattan_distance(a, b):
    return sum(abs(val1-val2) for val1, val2 in zip(a,b))

def Euclidean_distance(a, b):
    return np.linalg.norm(np.array(a)-np.array(b))

def cal_similarity(query_embeding, method):
  if method == "cosine_similarity":
      dataDict["similarities"] = dataDict.embeding.apply(lambda x: cosine_similarity(x, query_embeding))
      res = (
        dataDict.sort_values("similarities", ascending=False).head(1)
      )
      cosine_pred = res.dataSource.iloc[0]
      return cosine_pred

  elif method ==  "jaccard_similarity":
      dataDict["similarities"] = dataDict.dataString.apply(lambda x: jaccard_similarity(x, query_embeding))
      res = (
        dataDict.sort_values("similarities", ascending=False).head(1)
      )
      jaccard_pred = res.dataSource.iloc[0]
      return jaccard_pred

  elif method ==  "manhattan_distance":
      dataDict["similarities"] = dataDict.embeding.apply(lambda x: manhattan_distance(x, query_embeding))
      res = (
        dataDict.sort_values("similarities", ascending=True).head(1)
      )
      manhattan_pred = res.dataSource.iloc[0]
      return manhattan_pred

  elif method ==  "Euclidean_distance":
      dataDict["similarities"] = dataDict.embeding.apply(lambda x: Euclidean_distance(x, query_embeding))
      res = (
        dataDict.sort_values("similarities", ascending=True).head(1)
      )
      euclidean_pred = res.dataSource.iloc[0]
      return euclidean_pred

In [ ]:
querySimilarity["cosine_pred"] = querySimilarity.embeding.apply(lambda x : cal_similarity(x, "cosine_similarity"))
querySimilarity["jaccard_pred"] = querySimilarity.Query.apply(lambda x : cal_similarity(x, "jaccard_similarity"))
querySimilarity["manhattan_pred"] = querySimilarity.embeding.apply(lambda x : cal_similarity(x, "manhattan_distance"))
querySimilarity["euclidean_pred"] = querySimilarity.embeding.apply(lambda x : cal_similarity(x, "Euclidean_distance"))
querySimilarity.to_json(path+"querySimilarity.json")
querySimilarity

,Query,class,embeding,cosine_pred,jaccard_pred,manhattan_pred,euclidean_pred
0,What is the overall duration of the PDEV6800Z ...,Class Schedule,"[0.013200642541050911, -0.014171581715345383, ...",Administrative Issues and Policies,Class Schedule,Administrative Issues and Policies,Administrative Issues and Policies
1,How many weeks does the course span?,Class Schedule,"[0.007477969862520695, -0.0019203861011192203,...",Class Schedule,Instructor Contacts,Class Schedule,Class Schedule
2,What are the different sections available for ...,Class Schedule,"[-0.0027097552083432674, -0.003473019693046808...",Class Schedule,Instructor Contacts,Class Schedule,Class Schedule
3,How many modules are there in the course?,Class Schedule,"[9.099176531890407e-05, -0.0010510976426303387...",Class Schedule,Instructor Contacts,Class Schedule,Class Schedule
4,What is the general structure of the course sc...,Class Schedule,"[0.0001531848538434133, -0.0038599621038883924...",Class Schedule,Instructor Contacts,Class Schedule,Class Schedule
...,...,...,...,...,...,...,...
1095,Can you teach me how to juggle water?,irrelevant,"[-0.007677948568016291, -0.005169344134628773,...",Instructor Contacts,Instructor Contacts,Instructor Contacts,Instructor Contacts
1096,What's the best way to butter toast underwater?,irrelevant,"[-0.005880190059542656, -0.0015551196411252022...",Instructor Contacts,Instructor Contacts,Instructor Contacts,Instructor Contacts
1097,How do I grow a money tree?,irrelevant,"[0.004561420995742083, -0.03581124171614647, -...",Gamification,Instructor Contacts,Gamification,Gamification
1098,Why don't we call oranges 'yellows'?,irrelevant,"[0.021855326369404793, -0.02520665153861046, 0...",Instructor Contacts,Instructor Contacts,Instructor Contacts,Instructor Contacts


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
print("Accuracy of Cosine Similarity: ",
	accuracy_score(querySimilarity['class'], querySimilarity['cosine_pred']))
print("Classification Report of Cosine Similarity:\n",
      classification_report(querySimilarity['class'], querySimilarity['cosine_pred']))
print("Accuracy of jaccard_similarity: ",
	accuracy_score(querySimilarity['class'], querySimilarity['jaccard_pred']))
print("Classification Report of jaccard_similarity:\n",
      classification_report(querySimilarity['class'], querySimilarity['jaccard_pred']))
print("Accuracy of manhattan_distance: ",
	accuracy_score(querySimilarity['class'], querySimilarity['manhattan_pred']))
print("Classification Report of manhattan_distance:\n",
      classification_report(querySimilarity['class'], querySimilarity['manhattan_pred']))
print("Accuracy of Euclidean_distance: ",
	accuracy_score(querySimilarity['class'], querySimilarity['euclidean_pred']))
print("Classification Report of Euclidean_distance:\n",
      classification_report(querySimilarity['class'], querySimilarity['euclidean_pred']))

Accuracy of Cosine Similarity:  0.48727272727272725
Classification Report of Cosine Similarity:
                                     precision    recall  f1-score   support

Administrative Issues and Policies       0.28      0.56      0.37       151
           Assessments and Grading       0.66      0.38      0.48       159
                    Class Schedule       0.39      0.99      0.56       151
      Course Content and Structure       0.60      0.02      0.04       140
                      Gamification       0.88      0.91      0.89       152
               Instructor Contacts       0.64      0.69      0.66       147
                        irrelevant       0.00      0.00      0.00       200

                          accuracy                           0.49      1100
                         macro avg       0.49      0.51      0.43      1100
                      weighted avg       0.47      0.49      0.41      1100

Accuracy of jaccard_similarity:  0.12545454545454546
Classificat

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classif

#Training the classification of query

In [4]:
org = pd.read_json(path+"most-updated-student-queries.json")

org["embeding"] = querySimilarity["embeding"]
org.to_json(path+"trainingQuery.json")


In [5]:
org

,Query,class,embeding
0,What is the overall duration of the PDEV6800Z ...,Class Schedule,"[0.0132006425, -0.0141715817, 0.00358765500000..."
1,How many weeks does the course span?,Class Schedule,"[0.0074779699, -0.0019203861, 0.0194210075, -0..."
2,What are the different sections available for ...,Class Schedule,"[-0.0027097552, -0.0034730197, 0.0012607195, -..."
3,How many modules are there in the course?,Class Schedule,"[9.09918e-05, -0.0010510976, 0.0036951634, -0...."
4,What is the general structure of the course sc...,Class Schedule,"[0.0001531849, -0.0038599621000000002, 0.00647..."
...,...,...,...
1095,Can you teach me how to juggle water?,irrelevant,"[-0.007677948600000001, -0.0051693441, 0.00894..."
1096,What's the best way to butter toast underwater?,irrelevant,"[-0.0058801901, -0.0015551196, 0.0335660838, 0..."
1097,How do I grow a money tree?,irrelevant,"[0.004561421, -0.0358112417, -0.0048262775, -0..."
1098,Why don't we call oranges 'yellows'?,irrelevant,"[0.0218553264, -0.0252066515, 0.0213598013, -0..."


In [6]:
import pandas as pd
path = '/content/drive/MyDrive/Colab Notebooks/chatbot/'
df = pd.read_json(path+"trainingQuery.json")
df

,Query,class,embeding
0,What is the overall duration of the PDEV6800Z ...,Class Schedule,"[0.0132006425, -0.0141715817, 0.00358765500000..."
1,How many weeks does the course span?,Class Schedule,"[0.0074779699, -0.0019203861, 0.0194210075, -0..."
2,What are the different sections available for ...,Class Schedule,"[-0.0027097552, -0.0034730197, 0.0012607195, -..."
3,How many modules are there in the course?,Class Schedule,"[9.09918e-05, -0.0010510976, 0.0036951634, -0...."
4,What is the general structure of the course sc...,Class Schedule,"[0.0001531849, -0.0038599621000000002, 0.00647..."
...,...,...,...
1095,Can you teach me how to juggle water?,irrelevant,"[-0.007677948600000001, -0.0051693441, 0.00894..."
1096,What's the best way to butter toast underwater?,irrelevant,"[-0.0058801901, -0.0015551196, 0.0335660838, 0..."
1097,How do I grow a money tree?,irrelevant,"[0.004561421, -0.0358112417, -0.0048262775, -0..."
1098,Why don't we call oranges 'yellows'?,irrelevant,"[0.0218553264, -0.0252066515, 0.0213598013, -0..."


In [7]:
# Importing the required libraries
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from sklearn import svm
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier

X = np.array(df.embeding.tolist())
y = df['class']

# splitting X and y into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
	X, y, test_size=0.2, random_state=1)

# GAUSSIAN NAIVE BAYES
gnb = GaussianNB()
gnb.fit(X_train, y_train)
gnb_pred = gnb.predict(X_test)
# print the performance metrics
print("Accuracy of Gaussian Naive Bayes: ",
	accuracy_score(y_test, gnb_pred))
print("Classification Report of Gaussian Naive Bayes:\n",
      classification_report(y_test, gnb_pred))

# DECISION TREE CLASSIFIER
dt = DecisionTreeClassifier(random_state=0)
dt.fit(X_train, y_train)
dt_pred = dt.predict(X_test)
# print the performance metrics
print("Accuracy of Decision Tree Classifier: ",
	accuracy_score(y_test, dt_pred))
print("Classification Report of Decision Tree Classifier:\n",
      classification_report(y_test, dt_pred))

# SUPPORT VECTOR MACHINE(linear)
svm_clf = svm.SVC(kernel='linear') # Linear Kernel
svm_clf.fit(X_train, y_train)
svm_clf_pred = svm_clf.predict(X_test)
# print the performance metrics
print("Accuracy of Support Vector Machine(linear): ",
	accuracy_score(y_test, svm_clf_pred))
print("Classification Report of Support Vector Machine(linear):\n",
      classification_report(y_test, svm_clf_pred))

# SUPPORT VECTOR MACHINE(poly)
svm_clf_poly = svm.SVC(kernel='poly') # Linear Kernel
svm_clf_poly.fit(X_train, y_train)
svm_clf_poly_pred = svm_clf_poly.predict(X_test)
# print the performance metrics
print("Accuracy of Support Vector Machine(poly): ",
	accuracy_score(y_test, svm_clf_poly_pred))
print("Classification Report of Support Vector Machine(poly):\n",
      classification_report(y_test, svm_clf_poly_pred))

# KNN
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)
knn_pred = knn.predict(X_test)
# print the performance metrics
print("Accuracy of KNN: ",
	accuracy_score(y_test, knn_pred))
print("Classification Report of KNN:\n",
      classification_report(y_test, knn_pred))

# Ada Boost
adb = AdaBoostClassifier()
adb.fit(X_train,y_train)
adb_pred = adb.predict(X_test)
# print the performance metrics
print("Accuracy of Ada Boost: ",
	accuracy_score(y_test, adb_pred))
print("Classification Report of Ada Boost:\n",
      classification_report(y_test, adb_pred))

# Random Forest
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_classifier.fit(X_train, y_train)
rf_pred = rf_classifier.predict(X_test)
# print the performance metrics
print("Accuracy of Random Forest:",
  accuracy_score(y_test, rf_pred))
print("Classification Report of Random Forest:\n",
      classification_report(y_test, rf_pred))


Accuracy of Gaussian Naive Bayes:  0.8
Classification Report of Gaussian Naive Bayes:
                                     precision    recall  f1-score   support

Administrative Issues and Policies       0.55      0.76      0.64        29
           Assessments and Grading       0.70      0.68      0.69        31
                    Class Schedule       0.89      0.91      0.90        34
      Course Content and Structure       0.54      0.59      0.57        22
                      Gamification       1.00      0.89      0.94        35
               Instructor Contacts       1.00      0.84      0.92        32
                        irrelevant       0.94      0.84      0.89        37

                          accuracy                           0.80       220
                         macro avg       0.80      0.79      0.79       220
                      weighted avg       0.82      0.80      0.81       220

Accuracy of Decision Tree Classifier:  0.5772727272727273
Classification R

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


Accuracy of Ada Boost:  0.44545454545454544
Classification Report of Ada Boost:
                                     precision    recall  f1-score   support

Administrative Issues and Policies       0.17      0.17      0.17        29
           Assessments and Grading       0.32      0.29      0.31        31
                    Class Schedule       0.31      0.15      0.20        34
      Course Content and Structure       0.33      0.36      0.35        22
                      Gamification       0.96      0.71      0.82        35
               Instructor Contacts       0.38      0.84      0.52        32
                        irrelevant       0.79      0.51      0.62        37

                          accuracy                           0.45       220
                         macro avg       0.47      0.43      0.43       220
                      weighted avg       0.49      0.45      0.44       220

Accuracy of Random Forest: 0.8318181818181818
Classification Report of Random Fo

In [8]:
#save the best model
import pickle
traget_filename = 'svm_poly_model.sav'
pickle.dump(svm_clf_poly, open(path+traget_filename, 'wb'))
filename = 'finalized_model.sav'
pickle.dump(svm_clf_poly, open(path+filename, 'wb'))

In [12]:
import skops.io as sio
filename = 'finalized_model.skops'
obj = sio.dump(svm_clf_poly, open(path+filename, 'wb'))

In [10]:
svm_clf_poly.predict(X)

array(['Course Content and Structure', 'Class Schedule', 'Class Schedule',
       ..., 'irrelevant', 'irrelevant', 'irrelevant'], dtype=object)